## 1. Pretrained ResNet

In [1]:
from torchvision.models.resnet import resnet18
import torch
import torch.nn as nn

In [2]:
model = resnet18(pretrained=True)

/home/aicads/anaconda3/envs/yolo/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/aicads/anaconda3/envs/yolo/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [4]:
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=10)
)

## 2. 데이터 전처리 정의

In [5]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import Resize, RandomCrop, RandomHorizontalFlip
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

transforms = Compose([
    Resize((128,128)),
    RandomCrop((128,128), padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

## 3. 데이터 불러오기

In [6]:
training_data = CIFAR10(root='./', train=True, download=True, transform=transforms)
test_data = CIFAR10(root='./', train=False, download=True, transform=transforms)

train_loader = DataLoader(dataset=training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## 4. 모델 정의하기

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 5. torchsummary로 모델 살펴보기

In [8]:
from torchsummary import summary
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
       BasicBlock-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
             ReLU-14           [-1, 64,

## 6. 학습 루프 정의하기

In [9]:
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)
max_acc = 0

for epoch in range(30):
    iterator = tqdm.tqdm(train_loader)
    
    for data, label in iterator:
        optim.zero_grad()
        
        preds = model(data.to(device))
        
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch: {epoch+1} loss:{loss.item()}")
    
    with torch.no_grad():
        iterator = tqdm.tqdm(test_loader)

        acc = 0 
        
        for data, label in iterator:
            preds = model(data.to(device))
            
            output = preds.data.max(1)[1]
            corr = output.eq(label.to(device).data).sum().item()
            acc += corr
            
        
        acc = acc / len(test_data)
        print(f"Accuracy: {acc:.4f}")
        
        if max_acc < acc:
            max_acc = acc
            print("Save Pth!")
            torch.save(model.state_dict(), "ResNet.pth")
        

100%|██████████| 157/157 [00:05<00:00, 27.71it/s]


Accuracy: 0.9200
Save Pth!


100%|██████████| 157/157 [00:07<00:00, 22.27it/s]


Accuracy: 0.9278
Save Pth!


100%|██████████| 157/157 [00:06<00:00, 22.71it/s]


...

Accuracy: 0.9372


100%|██████████| 157/157 [00:06<00:00, 22.49it/s]


Accuracy: 0.9426


100%|██████████| 157/157 [00:06<00:00, 23.03it/s]

Accuracy: 0.9404
